In [1]:
# load environment
import Pkg; Pkg.activate(".")

  Activating project at `~/Documents/PhD/MyPublications/2025/SISSI_II_Local_Bubble_Age/SISSI-II-Local-Bubble`


In [2]:
Pkg.instantiate();

Precompiling project...
  ✓ LaTeXStrings
  ✓ PtrArrays
  ✓ DefineSingletons
  ✓ InverseFunctions
  ✓ CompositionsBase
  ✓ ArgCheck
  ✓ Compat
  ✓ Requires
  ✓ ConstructionBase
  ✓ JLLWrappers
  ✓ OrderedCollections
  ✓ InitialValues
  ✓ AliasTables
  ✓ ConstructionBase → ConstructionBaseLinearAlgebraExt
  ✓ Compat → CompatLinearAlgebraExt
  ✓ Adapt
  ✓ IrrationalConstants
  ✓ Conda
  ✓ CFITSIO_jll
  ✓ Baselet
  ✓ Tables
  ✓ CompositionsBase → CompositionsBaseInverseFunctionsExt
  ✓ InverseFunctions → InverseFunctionsTestExt
  ✓ Adapt → AdaptSparseArraysExt
  ✓ ColorTypes
  ✓ LogExpFunctions
  ✓ CFITSIO
  ✓ InverseFunctions → InverseFunctionsDatesExt
  ✓ Referenceables
  ✓ LogExpFunctions → LogExpFunctionsInverseFunctionsExt
  ✓ MacroTools
  ✓ DataStructures
  ✓ FITSIO
  ✓ SortingAlgorithms
  ✓ Setfield
  ✓ Healpix
  ✓ ColorVectorSpace
  ✓ SplittablesBase
  ✓ Accessors
  ✓ StatsBase
  ✓ Accessors → LinearAlgebraExt
  ✓ Colors
  ✓ Accessors → TestExt
  ✓ BangBang
  ✓ BangBang → BangBangT

# Packages & Includes

In [3]:
using FITSIO
using StatsBase                         
using ThreadsX
using PyCall, PyPlot, ColorSchemes, Colors
using Healpix
using Peaks
using LinearAlgebra
ll = pyimport("labellines")

PyObject <module 'labellines' from '/home/leonard/miniconda3/lib/python3.9/site-packages/labellines/__init__.py'>

# Constants

## Numerical Constants

In [3]:
const Nside = 256            # Healpix resolution
const dΩ    = 4π / Nside^2 / 12 # Solid angle element of each Healpix pixel
const TINY = nextfloat(0.0)

5.0e-324

## Physical Constants

In [4]:
# Some physical constants (in cgs units)
const mH   = 1.67262192e-24        # proton/hydrogen mass
const X_H  = 0.76                  # primordial hydrodgen fraction
const Y_He = (1 - X_H) / X_H / 4   # primordial abundance of helium
const μ    = (1 / X_H) * mH        # primordial mean atomic weight
const kB   = 1.380649e-16          # Boltzmann constant

1.380649e-16

## Units

In [5]:
# All units are in cgs unless otherwise specified
# Length
const μm  = 1e-4
const pc  = 3.086e18
const kpc = 1e3 * pc

# Time
const yr  = 3.155e7
const kyr = 1e3 * yr
const Myr = 1e6 * yr

# Mass
const Msol = 1.989e33

# Velocity
const km_s    = 1e5

# Density
const Dunit = 1653.19183 # Conversion factor between differential extinction & Hydrogen number density (O'Neill et al. 2024)

1653.19183

# Function Definitions

## I/O

In [6]:
"Load 3D dust maps into memory from FITS file"
function read_dust_map_file(path_data::String)
    # get fits file
    f = FITS(path_data * "/samples_healpix.fits")

    # get coordinates
    r = read(f[3], "radial pixel centers")

    # get bin width
    rb = read(f[4], "radial pixel boundaries")
    dr = [rb[i] - rb[i-1] for i in 2:length(rb)]

    # get sample dust maps
    samples = read(f[2])

    # get differential extinction in the central region
    dA_in = read(f[5]) / rb[1]

    close(f)

    # convert samples to a more practicable format (Array of Vectors)
    profiles = Array{Vector{Float32}}(undef, (size(samples)[1], size(samples)[3]))

    for ind in CartesianIndices(profiles)                                                                                                                                                                
        profiles[ind] = samples[ind[1], :, ind[2]]                                                                                                                                                       
    end

    return r, dr, profiles, dA_in
end

"Load mean 3D dust maps into memory from FITS file"
function read_mean_dust_map_file(path_data::String)
    # get fits file
    f = FITS(path_data * "/mean_and_std_healpix.fits")

    # get coordinates
    r = read(f[4], "radial pixel centers")

    # get bin width
    rb = read(f[5], "radial pixel boundaries")
    dr = [rb[i] - rb[i-1] for i in 2:length(rb)]

    # get sample dust maps
    map_mean = read(f[2])

    # get differential extinction in the central region
    dA_in_mean = read(f[6]) / rb[1]

    close(f)

    # convert samples to a more practicable format (Array of Vectors)
    profiles_mean = Array{Vector{Float32}}(undef, size(map_mean)[1])

    for ind in eachindex(profiles_mean)                                                                                                                                                                
        profiles_mean[ind] = map_mean[ind, :]                                                                                                                                                       
    end

    return r, dr, profiles_mean, dA_in_mean
end

read_mean_dust_map_file

## Utility

In [7]:
"Return N logarithmically spaced points between logx0 and logx1."
LogRange(logx0::Real, logx1::Real, N::Integer) = 10.0.^(LinRange(logx0, logx1, N))

"convert healpix pixel to angle"
get_angle(ihp::Integer; nside::Integer=Nside) = pix2ang(Map{Float64, NestedOrder}(nside), ihp)

"get coordinates in heliocentric Galactic cartesian coordinates"
get_coordinates_3d(θ::Real, ϕ::Real) = [sin(θ) * cos(ϕ), sin(θ) * sin(ϕ), cos(θ)]

"Check if a number is NaN or Inf."
isbad(x::Real) = isnan(x) || isinf(x)

"Check if an array has NaN or Inf values."
hasbad(a::Array{<:Number}) = sum(isbad.(a)) .> 0

hasbad

## Smoothing Operators

In [8]:
"""
    Return a linear smoothing operator to smooth an array with a Gaussian smoothing kernel, maintaining conserved total volume.
    Data is smoothed over a range of size hsmth.
    nbuf can be used to add contributions from a fictional buffer region.
    'mode' determines how boundaries are treated:
    - :linex  => linearly extrapolate data
    - :mirror => mirror data
    - :const  => extrapolate with constant value 
"""
function Gaussian_smoother(x::Vector{<:Real}, dV::Vector{<:Real}, hsmth::Int64; nbuf::Real=0, mode::Symbol=:linex, spacing::Symbol=:log, weighting=:const)
    # Number of bins in x direction
    Nx = length(x)

    # Add some buffer space to avoid spurious "edge-peaks"
    dx     = x[2] - x[1]
    Nbuf   = round(Int, nbuf * hsmth  / dx)
    x_buf  = spacing == :log ? [x[1]^2 / x[i] for i in 2:Nbuf]           : [x[1] - i * dx for i in 1:Nbuf-1]
    dV_buf = spacing == :log ? [(x[1] / x[i])^3 * dV[1] for i in 2:Nbuf] : [((xi+0.5dx)^3 - (xi-0.5dx)^3) / 3 for xi in x_buf]

    # check what weights we use
    weights     = weighting == :const ? 1.0 : dV
    weights_buf = weighting == :const ? 1.0 : dV_buf

    # get norm vectors
    norm_buf = [sum(exp.(-((x .- x_buf[i]) / hsmth).^2 / 2) .* weights) + sum(exp.(-((x_buf .- x_buf[i]) / hsmth).^2 / 2) .* weights_buf) for i in eachindex(x_buf)]
    norm     = [sum(exp.(-((x .- x[i]) / hsmth).^2 / 2) .* weights) + sum(exp.(-((x_buf .- x[i]) / hsmth).^2 / 2) .* weights_buf) for i in eachindex(x)]
    
    # output object
    smoother = zeros((Nx, Nx))

    for i in eachindex(x)
        # get radius
        u = (x .- x[i]) / hsmth

        wk  = @. exp(-u^2 / 2) .* weights ./ norm
        
        δwk = zeros(Nx)
        if Nbuf > 0
            u_buf    = (x_buf .- x[i]) / hsmth
            wk_buf   =  @. exp(-u_buf^2 / 2) .* weights_buf ./ norm_buf

            if mode == :linex
                if spacing == :log
                    δwk[1]       += sum(wk_buf .* (1.0 .+ x_buf / x[1]))
                    δwk[2:Nbuf] .-= wk_buf .* x_buf / x[1]
                else
                    δwk[1]       += 2 * sum(wk_buf)
                    δwk[2:Nbuf] .-= wk_buf
                end
            elseif mode == :mirror
                δwk[2:Nbuf] = wk_buf
            elseif mode == :const
                δwk[1] = sum(wk_buf)
            end
        end
        
        smoother[i, :] = wk + δwk
    end

    return smoother
end

"Short hand notation for Gaussian smoother with constant weights."
Gaussian_smoother(x::Vector{<:Real}, hsmth::Int64; nbuf::Real=0, mode::Symbol=:linex, spacing::Symbol=:log) = Gaussian_smoother(x, ones(size(x)), hsmth, nbuf=nbuf, mode=mode, spacing=spacing, weighting=:const)

Gaussian_smoother

In [9]:
"Return indices of snapshots at which time equals roughly certain values."
function get_samplepoints(time::Vector{<:Real}, timepoints::AbstractRange)
    return [findmin(abs.(time .- t))[2] for t in timepoints]
end

get_samplepoints

## Peak Finding

In [10]:
"Find position of inner part of shell."
function find_inner_shell(ipeak::Integer, ρ::Vector{<:Real}, pks::Vector{<:Integer}; Δ::Real=0.5 * 2.04e-6)
    # compute cutoff value
    ρ_cut = ρ[ipeak] - Δ

    # get set of peak positions AFTER initial peak
    pks_out = pks[pks .< ipeak]
    pk_next = length(pks_out) > 0 ? pks_out[end] : nothing

    i0 = isnothing(pk_next) ? 1 : pk_next

    # find first position where ρ < ρ_cut
    i_cut = findlast(dA -> dA < ρ_cut, ρ[1:ipeak-1]) + 1
    i_cut = i_cut == ipeak ? i_cut - 1 : i_cut

    if isnothing(pk_next) || pk_next < i_cut
        return i_cut
    else
        return pk_next + findminima(ρ[pk_next:ipeak]).indices[1] - 1
    end
end

"Find position of outer part of shell."
function find_outer_shell(ipeak::Integer, ρ::Vector{<:Real}, pks::Vector{<:Integer}; Δ::Real=0.5 * 2.04e-6)
    # compute cutoff value
    ρ_cut = ρ[ipeak] - Δ

    # get set of peak positions AFTER initial peak
    pks_out = pks[pks .> ipeak]
    pk_next = length(pks_out) > 0 ? pks_out[1] : nothing

    # find first position where ρ < ρ_cut
    i_cut = findfirst(dA -> dA < ρ_cut, ρ[ipeak:end])
    i_cut = isnothing(i_cut) ? length(ρ) : i_cut + ipeak - 2

    i_cut = i_cut == ipeak ? i_cut + 1 : i_cut

    if isnothing(pk_next) || pk_next > i_cut
        return i_cut
    else
        return ipeak + findminima(ρ[ipeak:pk_next]).indices[1] - 1
    end
end

find_outer_shell

## Shape Matrix

In [82]:
"Return the geometrical center and shape matrix of the structure."
function get_geometry(R_shell::Matrix{<:Real})
    # compute time-dependent geometry
    x_c = Matrix{Float64}(undef, 3, 12)
    I_s = Array{Float64}(undef, 3, 3, 12)

    for is in 1:12
        # get coordinates and volumes
        x_shell  = ThreadsX.map(ihp -> R_shell[ihp, is] * get_coordinates_3d(get_angle(ihp)...), eachindex(R_shell[:, is]))
        dV       = ThreadsX.map(r -> r^3 / 3, R_shell[:, is]) 

        # compute volume center
        x_c[:, is] = 0.75 * sum(x_shell .* dV) / sum(dV)
        
        # shift to center and recompute volumes
        dx = ThreadsX.map(x -> x - x_c[:, is], x_shell)
        dV = ThreadsX.map(δx -> (δx'δx)^1.5 / 3, dx)

        # compute shape tensor
        I_s[:, :, is] = 0.6 * sum(dV .* ThreadsX.map(δx -> δx'δx * I - δx * δx', dx)) / sum(dV)
    end        

    return x_c, I_s
end

"Return the geometrical center and shape matrix of the structure."
function get_geometry(R_shell::Vector{<:Real})
    # compute time-dependent geometry
    x_c = Vector{Float64}(undef, 3)
    I_s = Matrix{Float64}(undef, 3, 3)

    # get coordinates and volumes
    x_shell  = ThreadsX.map(ihp -> R_shell[ihp] * get_coordinates_3d(get_angle(ihp)...), eachindex(R_shell))
    dV       = ThreadsX.map(r -> r^3 / 3, R_shell) 

    # compute volume center
    x_c = 0.75 * sum(x_shell .* dV) / sum(dV)
    
    # shift to center and recompute volumes
    dx = ThreadsX.map(x -> x - x_c, x_shell)
    dV = ThreadsX.map(δx -> (δx'δx)^1.5 / 3, dx)

    # compute shape tensor
    I_s[:, :] = 0.6 * sum(dV .* ThreadsX.map(δx -> δx'δx * I - δx * δx', dx)) / sum(dV)     

    return x_c, I_s
end

"return ellipsoid radii and principal directions of 3x3 matrix."
function get_ellipsoid(M::Matrix{<:Real})
    if hasbad(M)
        return fill(NaN, 3), fill(fill(NaN, 3), 3)
    else
        eig  = eigen(M)
        R    = sqrt.(2.5 * tr(M) .- 5 * eig.values)
        vecs = [eig.vectors[:, i] for i in 1:3]
        return R, vecs
    end
end

"Check if l and be are in the forward direction (positive-x) and if not mirror them."
function forward_frame(l::Real, b::Real)
    if -90 < l < 90
        return l, b
    elseif l < -90
        return l + 180.0, -b
    else
        return l - 180.0, -b
    end
end

"Return the shape parameters and directors for a geometrical object described by a center and a shape matrix."
function get_geometry(x_c::Vector{<:Real}, I_s::Matrix{<:Real}; x_origin::Vector{<:Real}=[8000, 0, 0], rot::Integer=-1)
    # get shape eigenvalues and directions
    abc, dirs = get_ellipsoid(I_s)

    # get vectors relative to Galactic center
    x_center = x_c - x_origin

    # get coordinate system
    e_φ  = normalize(cross([0, 0, rot], x_center))
    e_r  = normalize(cross([0, 0, rot], e_φ))

    # Get axis ratios & effective radius
    minor = abc[3] / abc[1]
    major = abc[2] / abc[1]
    R_eff = prod(abc)^(1/3)        

    # get angles (minor)
    if dirs[3][3] < 0 dirs[3] *= -1 end
    cosθ_minor = dirs[3][3]
    φ_minor    = atan(e_r'dirs[3], e_φ'dirs[3]) * 180/π
    ϕ_minor    = forward_frame(φ_minor, acosd(cosθ_minor))[1]

    # get angles (major)
    if dirs[1][3] < 0 dirs[1] *= -1 end
    cosθ_major = dirs[1][3]
    φ_major    = atan(e_r'dirs[1], e_φ'dirs[1]) * 180/π
    ϕ_major    = forward_frame(φ_major, acosd(cosθ_major))[1]

    return R_eff, minor, cosθ_minor, ϕ_minor, major, cosθ_major, ϕ_major
end

get_geometry

# Loading the data

In [129]:
# Enter here the path to the data directory, where the FITS files (samples_healpix.fits & mean_and_std_healpix.fits) are located
path_data = "path/to/data"

"path/to/data"

## Sample Maps

In [13]:
r, dr, profiles, dA_in = read_dust_map_file(path_data);

In [37]:
#  compute volume element
dV = r.^2 .* dr

516-element Vector{Float32}:
 1831.72
 1862.8799
 1894.5472
 1926.8042
 1959.549
 1992.9021
 2026.7595
 2061.2837
 2096.2903
 2131.9846
 2168.2178
 2205.1213
 2242.6228
    ⋮
    9.003066f6
    9.156063f6
    9.311883f6
    9.470228f6
    9.631315f6
    9.795186f6
    9.961708f6
    1.0131107f7
    1.0303429f7
    1.0478723f7
    1.0657036f7
    1.0838043f7

## Mean Map

In [15]:
r_mean, dr_mean, profiles_mean, dA_in_mean = read_mean_dust_map_file(path_data); 

In [16]:
#  compute volume element
dV_mean = r_mean.^2 .* dr_mean

516-element Vector{Float32}:
 1831.72
 1862.8799
 1894.5472
 1926.8042
 1959.549
 1992.9021
 2026.7595
 2061.2837
 2096.2903
 2131.9846
 2168.2178
 2205.1213
 2242.6228
    ⋮
    9.003066f6
    9.156063f6
    9.311883f6
    9.470228f6
    9.631315f6
    9.795186f6
    9.961708f6
    1.0131107f7
    1.0303429f7
    1.0478723f7
    1.0657036f7
    1.0838043f7

# Analysis

In [17]:
# Analysis Parameters
P_min   = 2.04e-6  # Minimum peak prominence 
p_shell = 0.5      # Fraction of peak prominence at which shell edges are placed

0.5

## This Work

In [18]:
# Compute Smoothing Operator
W_smooth = Gaussian_smoother(r, dV, 9, nbuf=2, mode=:linex, spacing=:log, weighting=:volume);

### Sample Maps

In [19]:
# smooth profiles
profiles_smth = ThreadsX.map(u -> W_smooth * u, profiles);

In [20]:
# get peaks
peaks = ThreadsX.map(u -> findmaxima(u), profiles_smth);

In [21]:
# Filter peaks based on prominence criterion
peaks_shell = ThreadsX.map(u -> peakproms(u, min=P_min)[1][1], peaks);
prom_shell  = ThreadsX.map(u -> peakproms(u, min=P_min)[4][1], peaks);

In [29]:
# Locate outer shell boundary
outer_shell = ThreadsX.map((u, ρ, pks, P) -> find_outer_shell(u, ρ, pks.indices, Δ=p_shell * P), peaks_shell, profiles_smth, peaks, prom_shell);

#### Observables

##### Geometry

In [30]:
# Distance to peak and outer shell boundary
R_peak = ThreadsX.map(u -> r[u], peaks_shell);
R_out  = ThreadsX.map(u -> r[u], outer_shell);

In [44]:
# Compute shape tensor & volume weighted center (Computationally expensive!)
x_c, I_s = get_geometry(R_peak);

In [45]:
# Extract geometry observables from x_c and I_s
R_eff                 = zeros(12)
minor2major_ratio     = zeros(12)
cosθ_minor            = zeros(12)
ϕ_minor               = zeros(12)
semimajor2major_ratio = zeros(12)
cosθ_major            = zeros(12)
ϕ_major               = zeros(12)

for i in 1:12
    R_eff[i], minor2major_ratio[i], cosθ_minor[i], ϕ_minor[i], semimajor2major_ratio[i], cosθ_major[i], ϕ_major[i] = get_geometry(x_c[:, i], I_s[:, :, i])
end

In [46]:
# get statistics and print them in a nice format
mn, σ = mean_and_std(R_eff)
println("R_eff = $(round(mn, digits=1)) ± $(round(σ, digits=1)) pc")
mn, σ = mean_and_std(minor2major_ratio)
println("minor-axis-ratio = $(round(mn, digits=3)) ± $(round(σ, digits=3))")
mn, σ = mean_and_std(cosθ_minor)
println("cosθ_minor = $(round(mn, digits=3)) ± $(round(σ, digits=3))")
mn, σ = mean_and_std(ϕ_minor)
println("ϕ_minor = $(round(mn, digits=1)) ± $(round(σ, digits=1))")

mn, σ = mean_and_std(semimajor2major_ratio)
println("major-axis-ratio = $(round(mn, digits=3)) ± $(round(σ, digits=3))")
mn, σ = mean_and_std(cosθ_major)
println("cosθ_major = $(round(mn, digits=3)) ± $(round(σ, digits=3))")
mn, σ = mean_and_std(ϕ_major)
println("ϕ_major = $(round(mn, digits=0)) ± $(round(σ, digits=0))")

R_eff = 212.3 ± 1.0 pc
minor-axis-ratio = 0.469 ± 0.007
cosθ_minor = 0.607 ± 0.022
ϕ_minor = -66.1 ± 2.7
major-axis-ratio = 0.562 ± 0.011
cosθ_major = 0.665 ± 0.011
ϕ_major = -19.0 ± 1.0


##### Mass, Volume, Density, Momentum & Kinetic Energy

In [38]:
# Get mass up to each peak for each direction
dM = ThreadsX.map((u, ρ0, ρ) -> ρ0 * r[1]^3 / 3 + sum(ρ[1:u] .* dV[1:u]), outer_shell, dA_in, profiles) * dΩ * Dunit * μ * pc^3 / Msol;
δV = dΩ / 3 * (R_out * pc).^3;
dp = ThreadsX.map((u, ρ0, ρ) -> ρ0 * r[1]^4 / 4 + sum(ρ[1:u] .* r[1:u] .* dV[1:u]), outer_shell, dA_in, profiles) * 0.5 * dΩ * Dunit * μ * pc^3 * pc / Myr / Msol / km_s;
dE = ThreadsX.map((u, ρ0, ρ) -> ρ0 * r[1]^5 / 5 + sum(ρ[1:u] .* r[1:u].^2 .* dV[1:u]), outer_shell, dA_in, profiles) * (1/8) * dΩ * Dunit * μ * pc^3 * (pc / Myr)^2;

In [39]:
# Total Mass, Volume and average density
M   = sum(dM, dims=(1,));
V   = sum(δV, dims=(1,));
ρ_V = M ./ V * Msol / μ;
p = sum(dp, dims=(1,));
E = sum(dE, dims=(1,));

In [41]:
# get statistics and print them in a nice format
mn, σ = mean_and_std(M / 1e5) 
println("M_swept = ($(round(mn, digits=2)) ± $(round(σ, digits=2))) × 10⁵ Msol")
mn, σ = mean_and_std(V / 1e7 / pc^3)
println("Volume = ($(round(mn, digits=2)) ± $(round(σ, digits=2))) × 10⁷ pc³")
mn, σ = mean_and_std(ρ_V)
println("⟨ρ⟩ = $(round(mn, digits=3)) ± $(round(σ, digits=3)) amu/cc")
mn, σ = mean_and_std(p / 1e7)
println("p = ($(round(mn, digits=2)) ± $(round(σ, digits=2))) × 10⁷ Msol km/s × (t / Myr)⁻¹")
mn, σ = mean_and_std(E / 1e51)
println("E_kin = ($(round(mn, digits=1)) ± $(round(σ, digits=1))) × 10⁵¹ erg × (t / Myr)⁻²")

M_swept = (6.24 ± 0.07) × 10⁵ Msol
Volume = (3.89 ± 0.05) × 10⁷ pc³
⟨ρ⟩ = 0.494 ± 0.006 amu/cc
p = (5.92 ± 0.1) × 10⁷ Msol km/s × (t / Myr)⁻¹
E_kin = (62.8 ± 1.5) × 10⁵¹ erg × (t / Myr)⁻²


##### Alternate Way to compute Momentum & Energy: Thin Shell Approximation

In [34]:
# compute momentum by integrating rdM
p_thin = 0.5 * sum(dM .* R_out, dims=(1,)) * pc / Myr / km_s;
E_thin = 1/8 * sum(dM .* R_out.^2, dims=(1,)) * (pc / Myr)^2 * Msol;

In [43]:
# get statistics and print them in a nice format
mn, σ = mean_and_std(p_thin / 1e7)
println("p = ($(round(mn, digits=2)) ± $(round(σ, digits=2))) × 10⁷ Msol km/s × (t / Myr)⁻¹")
mn, σ = mean_and_std(E_thin / 1e51)
println("E_kin = ($(round(mn, digits=1)) ± $(round(σ, digits=1))) × 10⁵¹ erg × (t / Myr)⁻²")

p = (6.63 ± 0.1) × 10⁷ Msol km/s × (t / Myr)⁻¹
E_kin = (76.9 ± 1.7) × 10⁵¹ erg × (t / Myr)⁻²


### Mean Map

In [62]:
# smooth profiles
profiles_smth_mean = ThreadsX.map(u -> W_smooth * u, profiles_mean);

In [63]:
# get peaks
peaks_mean = ThreadsX.map(u -> findmaxima(u), profiles_smth_mean);

In [64]:
# Filter peaks based on prominence criterion
peaks_shell_mean = ThreadsX.map(u -> peakproms(u, min=P_min)[1][1], peaks_mean);
prom_shell_mean  = ThreadsX.map(u -> peakproms(u, min=P_min)[4][1], peaks_mean);

In [65]:
# Locate outer and inner shell boundary
outer_shell_mean = ThreadsX.map((u, ρ, pks, P) -> find_outer_shell(u, ρ, pks.indices, Δ=p_shell * P), peaks_shell_mean, profiles_smth_mean, peaks_mean, prom_shell_mean);
inner_shell_mean = ThreadsX.map((u, ρ, pks, P) -> find_inner_shell(u, ρ, pks.indices, Δ=p_shell * P), peaks_shell_mean, profiles_smth_mean, peaks_mean, prom_shell_mean);

#### Observables

##### Geometry

In [66]:
# Distance to peak and outer shell boundary
R_peak_mean = ThreadsX.map(u -> r[u], peaks_shell_mean);
R_out_mean  = ThreadsX.map(u -> r[u], outer_shell_mean);
R_in_mean   = ThreadsX.map(u -> r[u], inner_shell_mean);

In [83]:
# Compute shape tensor & volume weighted center (Computationally expensive!)
x_c_mean, I_s_mean = get_geometry(R_peak_mean);

In [85]:
# Extract geometry observables from x_c and I_s
R_eff_mean, minor2major_ratio_mean, cosθ_minor_mean, ϕ_minor_mean, semimajor2major_ratio_mean, cosθ_major_mean, ϕ_major_mean = get_geometry(x_c_mean, I_s_mean);

In [87]:
# print results in a nice format
println("R_eff = $(round(R_eff_mean, digits=1)) pc")
println("minor-axis-ratio = $(round(minor2major_ratio_mean, digits=3))")
println("cosθ_minor = $(round(cosθ_minor_mean, digits=3))")
println("ϕ_minor = $(round(ϕ_minor_mean, digits=1))")
println("major-axis-ratio = $(round(semimajor2major_ratio_mean, digits=3))")
println("cosθ_major = $(round(cosθ_major_mean, digits=3))")
println("ϕ_major = $(round(ϕ_major_mean, digits=0))")

R_eff = 247.6 pc
minor-axis-ratio = 0.44
cosθ_minor = 0.303
ϕ_minor = -81.2
major-axis-ratio = 0.544
cosθ_major = 0.799
ϕ_major = -16.0


##### Mass, Volume, Density, Momentum & Kinetic Energy

In [67]:
# Get mass up to each peak for each direction
dM_mean = ThreadsX.map((u, ρ0, ρ) -> ρ0 * r[1]^3 / 3 + sum(ρ[1:u] .* dV[1:u]), outer_shell_mean, dA_in_mean, profiles_mean) * dΩ * Dunit * μ * pc^3 / Msol;
dV_mean = dΩ / 3 * (R_out_mean * pc).^3;
dp_mean = ThreadsX.map((u, ρ0, ρ) -> ρ0 * r[1]^4 / 4 + sum(ρ[1:u] .* r[1:u] .* dV[1:u]), outer_shell_mean, dA_in_mean, profiles_mean) * 0.5 * dΩ * Dunit * μ * pc^3 * pc / Myr / Msol / km_s;
dE_mean = ThreadsX.map((u, ρ0, ρ) -> ρ0 * r[1]^5 / 5 + sum(ρ[1:u] .* r[1:u].^2 .* dV[1:u]), outer_shell_mean, dA_in_mean, profiles_mean) * (1/8) * dΩ * Dunit * μ * pc^3 * (pc / Myr)^2;

In [68]:
# Total Mass, Volume and average density
M_mean   = sum(dM_mean);
V_mean   = sum(dV_mean);
ρ_V_mean = M_mean / V_mean * Msol / μ;
p_mean = sum(dp_mean);
E_mean = sum(dE_mean);

In [69]:
# get statistics and print them in a nice format
println("M_swept = $(round(M_mean / 1e5, digits=2)) × 10⁵ Msol")
println("Volume = $(round(V_mean / 1e7 / pc^3, digits=2)) × 10⁷ pc³")
println("⟨ρ⟩ = $(round(ρ_V_mean, digits=3)) amu/cc")
println("p = $(round(p_mean / 1e7, digits=2)) × 10⁷ Msol km/s × (t / Myr)⁻¹")
println("E_kin = $(round(E_mean / 1e51, digits=1)) × 10⁵¹ erg × (t / Myr)⁻²")

M_swept = 7.54 × 10⁵ Msol
Volume = 5.8 × 10⁷ pc³
⟨ρ⟩ = 0.4 amu/cc
p = 7.77 × 10⁷ Msol km/s × (t / Myr)⁻¹
E_kin = 90.6 × 10⁵¹ erg × (t / Myr)⁻²


##### Alternate Way to compute Momentum & Energy: Thin Shell Approximation

In [75]:
# compute momentum by integrating rdM
p_thin_mean  = 0.5 * sum(dM_mean  .* R_out_mean) * pc / Myr / km_s;
E_thin_mean  = 1/8 * sum(dM_mean  .* R_out_mean .^2) * (pc / Myr)^2 * Msol;

In [76]:
# get statistics and print them in a nice format
println("p = $(round(p_thin_mean / 1e7, digits=2)) × 10⁷ Msol km/s × (t / Myr)⁻¹")
println("E_kin = $(round(E_thin_mean / 1e51, digits=1)) × 10⁵¹ erg × (t / Myr)⁻²")

p = 8.74 × 10⁷ Msol km/s × (t / Myr)⁻¹
E_kin = 112.9 × 10⁵¹ erg × (t / Myr)⁻²


##### Mass and density within Shell

In [72]:
# Get mass up to each peak for each direction
dM_shell_mean = ThreadsX.map((u0, u1, ρ) -> sum(ρ[u0:u1] .* dV[u0:u1]), inner_shell_mean, outer_shell_mean, profiles_mean) * dΩ * Dunit * μ * pc^3 / Msol;
dV_shell_mean = dΩ / 3 * (R_out_mean.^3 - R_in_mean.^3) * pc^3;

In [73]:
# Total Mass, Volume and average density
M_shell_mean   = sum(dM_shell_mean);
V_shell_mean   = sum(dV_shell_mean);
ρ_shell_mean   = M_shell_mean / V_shell_mean * Msol / μ;

In [74]:
# get statistics and print them in a nice format
println("M_shell      = $(round(M_shell_mean / 1e5, digits=2)) × 10⁵ Msol")
println("Shell Volume = $(round(V_shell_mean / 1e7 / pc^3, digits=2)) × 10⁷ pc³")
println("⟨ρ⟩_shell    = $(round(ρ_shell_mean, digits=3)) amu/cc")

M_shell      = 6.46 × 10⁵ Msol
Shell Volume = 2.99 × 10⁷ pc³
⟨ρ⟩_shell    = 0.664 amu/cc


## O'Neill et al. 2024

In [125]:
# Get projection onto linearly spaced grid
r_ONeill  = 69:1:1244
ir        = get_samplepoints(r, r_ONeill)
dV_ONeill = r_ONeill.^2 * 1.0;

In [89]:
# Compute Smoothing Operator
W_smooth_ONeill = Gaussian_smoother([r_ONeill...], 7, nbuf=2, mode=:mirror, spacing=:linear);

### Sample Maps

In [107]:
# Sample profiles on linearly spaced grid
profiles_ONeill = ThreadsX.map(u -> u[ir], profiles);

In [108]:
# smooth profiles
profiles_smth_ONeill = ThreadsX.map(u -> W_smooth_ONeill * u, profiles_ONeill);

In [109]:
# get peaks
peaks_ONeill = ThreadsX.map(u -> findmaxima(u), profiles_smth_ONeill);

In [110]:
# Filter peaks based on prominence criterion
peaks_shell_ONeill = ThreadsX.map(u -> peakproms(u, min=P_min)[1][1], peaks_ONeill);
prom_shell_ONeill  = ThreadsX.map(u -> peakproms(u, min=P_min)[4][1], peaks_ONeill);

In [118]:
# Locate outer shell boundary
outer_shell_ONeill = ThreadsX.map((u, ρ, pks, P) -> find_outer_shell(u, ρ, pks.indices, Δ=p_shell * P), peaks_shell_ONeill, profiles_smth_ONeill, peaks_ONeill, prom_shell_ONeill);

#### Observables

##### Geometry

In [119]:
# Distance to peak and outer shell boundary
R_peak_ONeill = ThreadsX.map(u -> r_ONeill[u], peaks_shell_ONeill);
R_out_ONeill  = ThreadsX.map(u -> r_ONeill[u], outer_shell_ONeill);

In [126]:
# Compute shape tensor & volume weighted center (Computationally expensive!)
x_c_ONeill, I_s_ONeill = get_geometry(R_peak_ONeill);

In [127]:
# Extract geometry observables from x_c and I_s
R_eff_ONeill                 = zeros(12)
minor2major_ratio_ONeill     = zeros(12)
cosθ_minor_ONeill            = zeros(12)
ϕ_minor_ONeill               = zeros(12)
semimajor2major_ratio_ONeill = zeros(12)
cosθ_major_ONeill            = zeros(12)
ϕ_major_ONeill               = zeros(12)

for i in 1:12
    R_eff_ONeill[i], minor2major_ratio_ONeill[i], cosθ_minor_ONeill[i], ϕ_minor_ONeill[i], semimajor2major_ratio_ONeill[i], cosθ_major_ONeill[i], ϕ_major_ONeill[i] = get_geometry(x_c_ONeill[:, i], I_s_ONeill[:, :, i])
end

In [128]:
# get statistics and print them in a nice format
mn, σ = mean_and_std(R_eff_ONeill)
println("R_eff = $(round(mn, digits=1)) ± $(round(σ, digits=1)) pc")
mn, σ = mean_and_std(minor2major_ratio_ONeill)
println("minor-axis-ratio = $(round(mn, digits=3)) ± $(round(σ, digits=3))")
mn, σ = mean_and_std(cosθ_minor_ONeill)
println("cosθ_minor = $(round(mn, digits=3)) ± $(round(σ, digits=3))")
mn, σ = mean_and_std(ϕ_minor_ONeill)
println("ϕ_minor = $(round(mn, digits=1)) ± $(round(σ, digits=1))")

mn, σ = mean_and_std(semimajor2major_ratio_ONeill)
println("major-axis-ratio = $(round(mn, digits=3)) ± $(round(σ, digits=3))")
mn, σ = mean_and_std(cosθ_major_ONeill)
println("cosθ_major = $(round(mn, digits=3)) ± $(round(σ, digits=3))")
mn, σ = mean_and_std(ϕ_major_ONeill)
println("ϕ_major = $(round(mn, digits=0)) ± $(round(σ, digits=0))")

R_eff = 200.4 ± 1.2 pc
minor-axis-ratio = 0.463 ± 0.008
cosθ_minor = 0.664 ± 0.013
ϕ_minor = -60.1 ± 2.2
major-axis-ratio = 0.576 ± 0.016
cosθ_major = 0.654 ± 0.01
ϕ_major = -20.0 ± 1.0


##### Mass, Volume, Density, Momentum & Kinetic Energy

In [120]:
# Get mass up to each peak for each direction
dM_ONeill = ThreadsX.map((u, ρ0, ρ) -> ρ0 * r_ONeill[1]^3 / 3 + sum(ρ[1:u] .* dV_ONeill[1:u]), outer_shell_ONeill, dA_in, profiles_ONeill) * dΩ * Dunit * μ * pc^3 / Msol;
δV_ONeill = dΩ / 3 * (R_out_ONeill * pc).^3;
dp_ONeill = ThreadsX.map((u, ρ0, ρ) -> ρ0 * r_ONeill[1]^4 / 4 + sum(ρ[1:u] .* r_ONeill[1:u] .* dV_ONeill[1:u]), outer_shell_ONeill, dA_in, profiles_ONeill) * 0.5 * dΩ * Dunit * μ * pc^3 * pc / Myr / Msol / km_s;
dE_ONeill = ThreadsX.map((u, ρ0, ρ) -> ρ0 * r_ONeill[1]^5 / 5 + sum(ρ[1:u] .* r_ONeill[1:u].^2 .* dV_ONeill[1:u]), outer_shell_ONeill, dA_in, profiles_ONeill) * (1/8) * dΩ * Dunit * μ * pc^3 * (pc / Myr)^2;

In [121]:
# Total Mass, Volume and average density
M_ONeill   = sum(dM_ONeill, dims=(1,));
V_ONeill   = sum(δV_ONeill, dims=(1,));
ρ_V_ONeill = M_ONeill ./ V_ONeill * Msol / μ;
p_ONeill = sum(dp_ONeill, dims=(1,));
E_ONeill = sum(dE_ONeill, dims=(1,));

In [122]:
# get statistics and print them in a nice format
mn, σ = mean_and_std(M_ONeill / 1e5) 
println("M_swept = ($(round(mn, digits=2)) ± $(round(σ, digits=2))) × 10⁵ Msol")
mn, σ = mean_and_std(V_ONeill / 1e7 / pc^3)
println("Volume = ($(round(mn, digits=2)) ± $(round(σ, digits=2))) × 10⁷ pc³")
mn, σ = mean_and_std(ρ_V_ONeill)
println("⟨ρ⟩ = $(round(mn, digits=3)) ± $(round(σ, digits=3)) amu/cc")
mn, σ = mean_and_std(p_ONeill / 1e7)
println("p = ($(round(mn, digits=2)) ± $(round(σ, digits=2))) × 10⁷ Msol km/s × (t / Myr)⁻¹")
mn, σ = mean_and_std(E_ONeill / 1e51)
println("E_kin = ($(round(mn, digits=1)) ± $(round(σ, digits=1))) × 10⁵¹ erg × (t / Myr)⁻²")

M_swept = (5.06 ± 0.08) × 10⁵ Msol
Volume = (3.26 ± 0.05) × 10⁷ pc³
⟨ρ⟩ = 0.477 ± 0.005 amu/cc
p = (4.52 ± 0.11) × 10⁷ Msol km/s × (t / Myr)⁻¹
E_kin = (44.7 ± 1.8) × 10⁵¹ erg × (t / Myr)⁻²


##### Alternate Way to compute Momentum & Energy: Thin Shell Approximation

In [123]:
# compute momentum by integrating rdM
p_thin_ONeill = 0.5 * sum(dM_ONeill .* R_out_ONeill, dims=(1,)) * pc / Myr / km_s;
E_thin_ONeill = 1/8 * sum(dM_ONeill .* R_out_ONeill.^2, dims=(1,)) * (pc / Myr)^2 * Msol;

In [124]:
# get statistics and print them in a nice format
mn, σ = mean_and_std(p_thin_ONeill / 1e7)
println("p = ($(round(mn, digits=2)) ± $(round(σ, digits=2))) × 10⁷ Msol km/s × (t / Myr)⁻¹")
mn, σ = mean_and_std(E_thin_ONeill / 1e51)
println("E_kin = ($(round(mn, digits=1)) ± $(round(σ, digits=1))) × 10⁵¹ erg × (t / Myr)⁻²")

p = (5.04 ± 0.12) × 10⁷ Msol km/s × (t / Myr)⁻¹
E_kin = (54.3 ± 2.0) × 10⁵¹ erg × (t / Myr)⁻²


### Mean Maps

In [90]:
# Sample profiles on linearly spaced grid
profiles_ONeill_mean = ThreadsX.map(u -> u[ir], profiles_mean);

In [91]:
# smooth profiles
profiles_smth_ONeill_mean = ThreadsX.map(u -> W_smooth_ONeill * u, profiles_ONeill_mean);

In [92]:
# get peaks
peaks_ONeill_mean = ThreadsX.map(u -> findmaxima(u), profiles_smth_ONeill_mean);

In [93]:
# Filter peaks based on prominence criterion
peaks_shell_ONeill_mean = ThreadsX.map(u -> peakproms(u, min=P_min)[1][1], peaks_ONeill_mean);
prom_shell_ONeill_mean  = ThreadsX.map(u -> peakproms(u, min=P_min)[4][1], peaks_ONeill_mean);

In [94]:
# Locate outer shell boundary
outer_shell_ONeill_mean = ThreadsX.map((u, ρ, pks, P) -> find_outer_shell(u, ρ, pks.indices, Δ=p_shell * P), peaks_shell_ONeill_mean, profiles_smth_ONeill_mean, peaks_ONeill_mean, prom_shell_ONeill_mean);
inner_shell_ONeill_mean = ThreadsX.map((u, ρ, pks, P) -> find_inner_shell(u, ρ, pks.indices, Δ=p_shell * P), peaks_shell_ONeill_mean, profiles_smth_ONeill_mean, peaks_ONeill_mean, prom_shell_ONeill_mean);

#### Observables

##### Geometry

In [95]:
# Distance to peak and shell boundaries
R_peak_ONeill_mean = ThreadsX.map(u -> r_ONeill[u], peaks_shell_ONeill_mean);
R_out_ONeill_mean  = ThreadsX.map(u -> r_ONeill[u], outer_shell_ONeill_mean);
R_in_ONeill_mean   = ThreadsX.map(u -> r_ONeill[u], inner_shell_ONeill_mean);

In [104]:
# Compute shape tensor & volume weighted center (Computationally expensive!)
x_c_ONeill_mean, I_s_ONeill_mean = get_geometry(R_peak_ONeill_mean);

In [105]:
# Extract geometry observables from x_c and I_s
R_eff_ONeill_mean, minor2major_ratio_ONeill_mean, cosθ_minor_ONeill_mean, ϕ_minor_ONeill_mean, 
                   semimajor2major_ratio_ONeill_mean, cosθ_major_ONeill_mean, ϕ_major_ONeill_mean = get_geometry(x_c_ONeill_mean, I_s_ONeill_mean);


(243.8068565807388, 0.44053201832012595, 0.3185938530759416, -80.2408699152979, 0.5436665532210188, 0.790214011493372, -15.924013034350775)

In [106]:
# print results in a nice format
println("R_eff = $(round(R_eff_ONeill_mean, digits=1)) pc")
println("minor-axis-ratio = $(round(minor2major_ratio_ONeill_mean, digits=3))")
println("cosθ_minor = $(round(cosθ_minor_ONeill_mean, digits=3))")
println("ϕ_minor = $(round(ϕ_minor_ONeill_mean, digits=1))")
println("major-axis-ratio = $(round(semimajor2major_ratio_ONeill_mean, digits=3))")
println("cosθ_major = $(round(cosθ_major_ONeill_mean, digits=3))")
println("ϕ_major = $(round(ϕ_major_ONeill_mean, digits=0))")

R_eff = 243.8 pc
minor-axis-ratio = 0.441
cosθ_minor = 0.319
ϕ_minor = -80.2
major-axis-ratio = 0.544
cosθ_major = 0.79
ϕ_major = -16.0


##### Mass, Volume, Density, Momentum & Kinetic Energy

In [96]:
# Get mass up to each peak for each direction
dM_ONeill_mean = ThreadsX.map((u, ρ0, ρ) -> ρ0 * r_ONeill[1]^3 / 3 + sum(ρ[1:u] .* dV_ONeill[1:u]), outer_shell_ONeill_mean, dA_in_mean, profiles_ONeill_mean) * dΩ * Dunit * μ * pc^3 / Msol;
dV_ONeill_mean = dΩ / 3 * (R_out_ONeill_mean * pc).^3;
dp_ONeill_mean = ThreadsX.map((u, ρ0, ρ) -> ρ0 * r_ONeill[1]^4 / 4 + sum(ρ[1:u] .* r_ONeill[1:u] .* dV_ONeill[1:u]), outer_shell_ONeill_mean, dA_in_mean, profiles_ONeill_mean) * 0.5 * dΩ * Dunit * μ * pc^3 * pc / Myr / Msol / km_s;
dE_ONeill_mean = ThreadsX.map((u, ρ0, ρ) -> ρ0 * r_ONeill[1]^5 / 5 + sum(ρ[1:u] .* r_ONeill[1:u].^2 .* dV_ONeill[1:u]), outer_shell_ONeill_mean, dA_in_mean, profiles_ONeill_mean) * (1/8) * dΩ * Dunit * μ * pc^3 * (pc / Myr)^2;

In [97]:
# Total Mass, Volume and average density
M_ONeill_mean   = sum(dM_ONeill_mean);
V_ONeill_mean   = sum(dV_ONeill_mean);
ρ_V_ONeill_mean = M_ONeill_mean / V_ONeill_mean * Msol / μ;
p_ONeill_mean = sum(dp_ONeill_mean);
E_ONeill_mean = sum(dE_ONeill_mean);

In [98]:
# get statistics and print them in a nice format
println("M_swept = $(round(M_ONeill_mean / 1e5, digits=2)) × 10⁵ Msol")
println("Volume = $(round(V_ONeill_mean / 1e7, digits=2)) × 10⁷ pc³")
println("⟨ρ⟩ = $(round(ρ_V_ONeill_mean, digits=3)) amu/cc")
println("p = $(round(p_ONeill_mean / 1e7, digits=2)) × 10⁷ Msol km/s × (t / Myr)⁻¹")
println("E_kin = $(round(E_ONeill_mean / 1e51, digits=1)) × 10⁵¹ erg × (t / Myr)⁻²")

M_swept = 6.92 × 10⁵ Msol
Volume = 1.6082443355680308e56 × 10⁷ pc³
⟨ρ⟩ = 0.389 amu/cc
p = 7.06 × 10⁷ Msol km/s × (t / Myr)⁻¹
E_kin = 81.6 × 10⁵¹ erg × (t / Myr)⁻²


##### Alternate Way to compute Momentum & Energy: Thin Shell Approximation

In [99]:
# compute momentum by integrating rdM
p_thin_ONeill_mean  = 0.5 * sum(dM_ONeill_mean  .* R_out_ONeill_mean) * pc / Myr / km_s;
E_thin_ONeill_mean  = 1/8 * sum(dM_ONeill_mean  .* R_out_ONeill_mean .^2) * (pc / Myr)^2 * Msol;

In [100]:
# get statistics and print them in a nice format
println("p = $(round(p_thin_ONeill_mean / 1e7, digits=2)) × 10⁷ Msol km/s × (t / Myr)⁻¹")
println("E_kin = $(round(E_thin_ONeill_mean / 1e51, digits=1)) × 10⁵¹ erg × (t / Myr)⁻²")

p = 7.91 × 10⁷ Msol km/s × (t / Myr)⁻¹
E_kin = 100.7 × 10⁵¹ erg × (t / Myr)⁻²


##### Mass and density within Shell

In [101]:
# Get mass up to each peak for each direction
dM_shell_ONeill_mean = ThreadsX.map((u0, u1, ρ) -> sum(ρ[u0:u1] .* dV_ONeill[u0:u1]), inner_shell_ONeill_mean, outer_shell_ONeill_mean, profiles_ONeill_mean) * dΩ * Dunit * μ * pc^3 / Msol;
dV_shell_ONeill_mean = dΩ / 3 * (R_out_ONeill_mean.^3 - R_in_ONeill_mean.^3) * pc^3;

In [102]:
# Total Mass, Volume and average density
M_shell_ONeill_mean   = sum(dM_shell_ONeill_mean);
V_shell_ONeill_mean   = sum(dV_shell_ONeill_mean);
ρ_shell_ONeill_mean   = M_shell_ONeill_mean / V_shell_ONeill_mean * Msol / μ;

In [103]:
# get statistics and print them in a nice format
println("M_shell      = $(round(M_shell_ONeill_mean / 1e5, digits=2)) × 10⁵ Msol")
println("Shell Volume = $(round(V_shell_ONeill_mean / 1e7 / pc^3, digits=2)) × 10⁷ pc³")
println("⟨ρ⟩_shell    = $(round(ρ_shell_ONeill_mean, digits=3)) amu/cc")

M_shell      = 5.76 × 10⁵ Msol
Shell Volume = 2.64 × 10⁷ pc³
⟨ρ⟩_shell    = 0.671 amu/cc


# Plots

In [130]:
# Plotting Parameters
fontsize = 18
output_directory = "path/to/figures/"
model_colors = Dict("N1" => "gray", "N10" => "red", "N1x10" => "royalblue")

Dict{String, String} with 3 entries:
  "N1"    => "gray"
  "N1x10" => "royalblue"
  "N10"   => "red"

## Figure 1: Expansion of SNRs in SISSI

In [48]:
# SISSI Tracks
t_N1 = [0.0, 0.005525858086756399, 0.013424681993113858, 0.024901474379083664, 0.036288267532528735, 0.047132892644936794, 0.05758093480414203, 0.068858157713799, 0.07997320407349244, 0.09106835896698424, 0.10273001031209802, 0.11532443413020173, 0.12887746418978693, 0.14349925367530228, 0.1587341434772749, 0.17457430991872558, 0.20882736766846696, 0.3077188245124922, 0.39486111700234877, 0.4867244224755129, 0.5596861416380255, 0.6601053131551318, 0.7591230471891974, 0.832381871293733, 0.9297013153350696, 0.9528464076752974, 1.000150468525969, 1.5048360833360335, 1.562129023030845, 2.0194195926469884, 2.5126590052088607, 2.6478338634535654, 2.999429994334223, 3.5206083577280545, 4.020470640431396, 4.5371814042751115, 5.022154493131932, 5.0713388929202665, 5.520782868421909, 6.035281437894905, 6.511421768611199, 7.050339540169883, 7.541713481089828, 7.595643802961089, 8.025288431153914, 8.515130010326184, 9.014642915562206, 9.508023046640163, 9.94138441090954, 10.049691356656858]
R_N1 = [NaN NaN NaN; 11.949452247159423 11.707452547056976 12.106820259532121; 17.084197999424653 16.95752279109409 17.384163137266345; 21.024373650511304 20.464083069380646 21.36349757458934; 23.6759201318452 22.756691441258376 24.190270407160803; 25.54383825173809 24.49884496071695 26.41953052690791; 27.328532206889946 26.235035287184584 28.37016491645028; 29.413562469402574 28.233102999525187 30.251527085222072; 30.93667510145498 29.607277052229936 31.874523621684936; 32.70568475342088 31.323020703896916 33.411688227193864; 33.97684376962744 32.44944607771661 34.798562560395176; 35.21557034063258 33.54159593426167 36.11623387355788; 36.43702788286603 34.6225895926892 37.446385448061655; 37.65482469183388 35.71380519636173 38.79688915062127; 38.8379473796751 36.80551877237061 40.13128163349438; 39.997377826467556 37.89733833685114 41.438304072360076; 42.93583454910971 41.3868973390777 44.2415648663102; 48.94690082453921 47.11887467109412 50.71738606880585; 52.514889121020374 50.64687449373361 55.10733408134742; 56.50281435462293 54.71553300596856 59.36751665333161; 59.33270914571733 57.59440021848318 62.36615360359254; 62.86057753320067 61.11563886177953 66.05286082233332; 66.01829659203028 64.2141480914263 69.28695229855437; 68.24684600534322 66.24510068025306 71.47730818420236; 71.03446485943687 68.71420329445499 74.1689629769624; 71.6665185441403 69.27401311821252 74.77222385400344; 72.9409487232792 70.38932189225807 76.00629314714732; 83.34225759134587 80.2303694332671 88.15134802271513; 84.27043418123483 81.16123543549726 89.44818268455808; 92.8687059331746 87.9986513455353 97.95783559025365; 101.38000570813512 94.59108562391876 106.80080355393139; 102.9048398928382 96.15737901430303 108.86229829204979; 106.86457485134812 100.09132987988978 114.21353009925113; 116.97541966718303 107.37562657000052 139.7333533780365; 122.0326805597713 111.57495891220904 151.2930388689487; 128.1414691423944 120.26877040815977 178.31196856229317; 132.4226621110184 127.9278942948901 167.45560490329825; 132.84988282881648 128.33717278804096 168.42310546028605; 136.5414372057875 131.9690173702826 177.01482314312503; 140.4564545191182 135.9473886673089 186.48305452130705; 142.5209994909838 136.92407404028097 161.6845703408867; 146.23535642885628 140.82828619114065 167.5152691906474; 149.44399209610486 144.3484723407397 172.5564590946816; 149.0080070201523 142.17136820999687 150.24714462489592; 151.9396609321708 145.37568774190237 152.73445941175652; 155.31198886277554 151.41298671097246 178.01964062772083; 158.07363590143083 154.8923761117412 179.02854261695853; 160.62788462626648 158.19017722697726 180.02417272884344; 162.7536711731325 161.08855232731293 180.78960546570096; 164.36226979088835 162.03231026082798 203.75268630817635]

t_N10 = [0.0, 0.0044213479365527875, 0.010276612659355788, 0.01700244728079353, 0.024832596697544652, 0.03355861174562682, 0.04367297765251567, 0.05358904607216131, 0.06408199835717555, 0.07416067199382928, 0.08553185463875178, 0.09734483402768537, 0.11038911840818995, 0.12385779773755203, 0.13754437888551194, 0.15166157739219155, 0.21280545378329643, 0.29992005005075595, 0.40208608618658037, 0.49392823608940456, 0.565381931432538, 0.6659042909676595, 0.7404841240453318, 0.8380062640911473, 0.9113357914517876, 1.0054394452197815, 1.4827818755328637, 1.511240451457575, 1.568648177627579, 2.0266627011655953, 2.5204054950063055, 3.00803128773034, 3.3185449351734215, 3.5303802485094113, 4.028984741158654, 4.545994533096546, 5.0312233729758615, 5.53013316636557, 6.04495521953555, 6.520963777283826, 7.043998610017217, 7.511466810434236, 8.032199268592695, 8.498847705216821, 9.018372971132386, 9.541488498393846, 10.003757446902432]
R_N10 = [NaN NaN NaN; 16.859639384761003 16.71618904508194 17.02945458012564; 23.825764643830926 23.66753889496927 23.9141198685182; 28.746173281841656 28.490320194556787 28.829508796158777; 32.52823804941049 31.628978376940562 32.94637345450656; 36.028601586937995 35.07489762180408 36.558694339949554; 38.670384350098814 37.30412115280356 39.663041103331665; 41.33423964586256 39.99130096045256 42.42736449449264; 44.2019850804089 42.56165048029055 45.23485183092643; 45.85532688602347 44.60479356366085 47.6429943901783; 47.98970978916144 46.94323939804904 50.2783128069189; 49.95717445508485 49.22981549824961 52.73235813264013; 52.370558908484405 51.61178943757698 55.514710204221565; 54.372703376084786 53.78141141652195 57.88040292265602; 56.32199312895264 55.651570662639756 59.19008393469353; 58.41003339767093 57.56514865212047 61.34878894243251; 66.36330516441687 64.57218242735594 69.51095013937167; 78.0237315071794 74.33165753105122 79.36056986699256; 87.4579885667475 83.27618880458627 87.96962221705125; 94.47745931666863 90.76529546948012 94.8484953645611; 99.08856749072612 94.52223394652955 100.65588105195921; 105.18896233845632 102.07074422850778 108.37383868406332; 110.64752163144573 106.00049354485371 113.75376510987039; 117.11556863638873 110.87683553706535 121.39311988497218; 118.82054641260979 107.92796802730005 126.93313202236288; 123.3317227848296 112.07080517434561 133.7938235657137; 143.01126404553327 130.17941570571065 153.6079866090692; 144.17075737675512 131.55310868002883 155.02425329224585; 146.38315139854515 133.82805448741618 157.69077976161353; 158.8758264577417 148.12118565393007 178.46783330866663; 168.5934663440369 161.5826534319196 187.98010467739505; 180.651246562469 175.31940129527516 203.26054277949657; 187.93937648246407 183.7387375510035 212.09650420185886; 193.42605363424053 189.07763586059065 217.8240108159825; 206.35636677860612 200.7209737017809 230.41455327952065; 218.4909790466904 211.91079102905775 241.78463274978736; 228.7221187921295 221.64229208039455 251.004420612756; 240.32850511886767 232.57573775563276 262.6530866747212; 249.30371582232223 241.79723241839162 274.7675824851983; 257.09920389942107 249.81238785801025 283.98189762192123; 265.2796384473777 258.15017969773123 291.3520630264149; 270.28567476609635 259.4154217972268 302.5785296651427; 287.74990463851793 275.3055396704747 318.4810238241755; 293.43300105428716 280.6869794710363 327.67326455759064; 341.01104619566894 298.53947251161327 626.8340655480441; 350.95279810577387 304.26342133019364 638.6942702061505; 385.0566067435961 341.71334878343566 835.7307662250464]

t_N1x10 = [0.0, 0.005525858086756399, 0.013424681993113858, 0.024901474379083664, 0.036288267532528735, 0.047132892644936794, 0.05758093480414203, 0.068858157713799, 0.07997320407349244, 0.09106835896698424, 0.10273001031209802, 0.11532443413020173, 0.12887746418978693, 0.14349925367530228, 0.1587341434772749, 0.17457430991872558, 0.19137867684443918, 0.20882736766846696, 0.22682486146604633, 0.2456790495308271, 0.2656699327505431, 0.2865035619220939, 0.3077188245124922, 0.4172301249121068, 0.510898049680113, 0.5596861416380255, 0.6857426425347739, 0.7834162198041471, 0.8572971796119683, 0.9528464076752974, 0.97656473766591, 1.000150468525969, 1.0115836648377727, 1.1079209262922871, 1.2088513630620186, 1.3123223447456092, 1.4127394240381046, 1.507659752205289, 1.9976670112861594, 2.0294776709205005, 2.0528840085686975, 2.102128550701235, 2.2032677278780715, 2.301720995184156, 2.4019023125619396, 2.500121892958679, 2.996894486995912, 3.023033982680257, 3.0836140504370086, 3.1044195174878157, 3.2104101706097823, 3.3088228074877923, 3.4000963258347325, 3.5039581542626865, 4.002118732147179, 4.026983280732671, 4.052368562765569, 4.103539951778044, 4.2067053775791905, 4.301077656704888, 4.401499487967776, 4.510340356591946, 4.839617359844025, 5.000159219340132, 5.024978730467529, 5.100797692374476, 5.2019813563720465, 5.313484055498141, 5.409284495606735, 5.502336888555106, 5.998065367804782, 6.024844822777571, 6.103657286887735, 6.201934053852153, 6.301718427565015, 6.411890720853098, 6.509361899428174, 6.997143378437413, 7.0247325587542955, 7.105575746029767, 7.193990167041049, 7.204921173785367, 7.300713403906202, 7.416559464853037, 7.509947543364291, 8.011254964890613, 8.039859286085969, 8.10289237098259, 8.207848808312317, 8.308966844376393, 8.406303367929796, 8.516358275365652, 9.017580671560326, 9.047861731251048, 9.109152095272696, 9.158057850972897, 9.203112221112821, 9.303892249132392, 9.408333989262557, 9.507139654682794, 9.971800403579063, 10.0008417683986]
R_N1x10 = [NaN NaN NaN; 11.949452247159423 11.707452547056976 12.106820259532121; 17.084197999424653 16.95752279109409 17.384163137266345; 21.024373650511304 20.464083069380646 21.36349757458934; 23.6759201318452 22.756691441258376 24.190270407160803; 25.54383825173809 24.49884496071695 26.41953052690791; 27.328532206889946 26.235035287184584 28.37016491645028; 29.413562469402574 28.233102999525187 30.251527085222072; 30.93667510145498 29.607277052229936 31.874523621684936; 32.70568475342088 31.323020703896916 33.411688227193864; 33.97684376962744 32.44944607771661 34.798562560395176; 35.21557034063258 33.54159593426167 36.11623387355788; 36.43702788286603 34.6225895926892 37.446385448061655; 37.65482469183388 35.71380519636173 38.79688915062127; 38.8379473796751 36.80551877237061 40.13128163349438; 39.997377826467556 37.89733833685114 41.438304072360076; 41.73213541208684 40.253799563761575 42.91956568912527; 42.93583454910971 41.3868973390777 44.2415648663102; 44.126793958964754 42.5091331300237 45.547270250668525; 45.31826862955484 43.644128567667416 46.83703141269371; 46.54152875884968 44.80444467513878 48.15267228144701; 47.76178060399155 45.973895025595894 49.45289705906378; 48.94690082453921 47.11887467109412 50.71738606880585; 53.5365461333643 51.674144923768054 56.20993960989501; 57.474433895316785 55.6948158550902 60.38723650259164; 59.33270914571733 57.59440021848318 62.36615360359254; 63.699116430229054 61.946760967523744 66.92624572144207; 66.76022928324291 64.92616035805261 70.02651706750456; 68.9768136761008 66.89949254325292 72.19570200638552; 71.6665185441403 69.27401311821252 74.77222385400344; 72.30526056071417 69.83770966671867 75.38707427609494; 72.9356027680166 70.38932189225807 75.98651917832188; 72.7798762731781 70.23382747789258 75.77159617449402; 75.2196988245606 72.36348254340345 78.09076785640603; 77.51854407615832 74.44958089073056 80.5432507883902; 79.73574776013173 76.59504192679177 83.26928656144895; 81.9685552151374 78.72194252040916 85.97220796958497; 84.09753515194843 80.74763423691275 88.55488764138101; 96.77088567845512 91.50107369233027 101.11050853475061; 99.45964912549573 92.64990474876207 102.41059525813345; 99.61469631216053 92.8456300167479 102.63045462959784; 100.87991111386782 94.05296284790063 104.20442579183583; 103.08974544921564 96.4085437710981 107.06994329025002; 105.0928610252912 98.66948620011776 109.66695736982179; 107.02352783105762 100.98068759614819 112.18727432987478; 108.8620711714309 103.3603253817606 114.59496201922563; 119.64428603160472 116.36083308645756 136.93886366587157; 120.10624359511966 116.84077053238069 137.74527846556595; 120.89222539913129 117.724787761108 139.25030976822566; 121.25803882215585 118.30463527152222 139.9040528031736; 123.10471435348632 121.22023189740278 143.26771589337113; 125.96954331716864 123.98743341146192 146.3624370112951; 129.01934618512544 126.35708415610272 149.23619188311542; 130.6147637886674 127.60118336569064 138.52132184595067; 144.41208677651093 136.72242648703303 153.94610754875995; 145.1336302223184 137.16890743053165 154.68080788100963; 145.8663618186609 137.6255626465129 155.41201791903316; 147.1167298772894 138.31347859692374 156.63691997165262; 149.5784898565461 140.17187567887137 160.42762358223138; 151.75461697865666 141.8495285626709 163.9486210201896; 152.01987163013138 144.01579711318348 162.70416077609303; 161.4163119213804 146.06949026253318 169.24952395967844; 164.0351214667549 151.57759974856424 194.73781926450195; 167.5688604956197 155.1566922028468 205.86043651085242; 162.43727876923856 155.1796239368301 198.81858111551404; 163.80789369255353 156.75483633963702 200.75604232252263; 171.70261486567546 159.64053426672288 220.8214914935716; 174.01804758985793 162.2067176601083 227.77593809256373; 181.59628150209682 165.00031852244578 294.3299228598594; 183.32268290922997 167.57434240067118 300.972045904252; 214.21581792674937 184.6555733151381 336.0123615427464; 204.50945976550696 185.44305776592654 275.56191950348176; 208.76470706961197 187.95364681511325 280.7974857119104; 194.75821256447614 190.92668342886404 241.32293793790683; 196.08346165939668 192.81824658591356 242.60319190090138; 198.86592550561454 196.74952623834687 250.50251258161646; 202.03510528601458 199.61549089711178 258.61664558526286; 222.7084125149712 211.53031618199972 277.59954027695704; 221.30319371491186 208.89833218197256 233.33276280203745; 224.26093568195517 210.15081398293253 238.365032687256; 223.08084423780684 210.33382244906522 303.57046099722083; 223.3493340064494 210.53480360179213 304.19596331114803; 225.99523777392923 212.2614835264159 309.54194428532696; 229.13478546990285 214.79948927656656 315.6997390462198; 231.57735371331222 216.90820007047904 320.44426611628205; 243.9541633677739 228.77785870380035 343.7475543319995; 237.2962631235672 228.71236770510322 416.728896076216; 238.6141163089407 229.91619230866576 419.2822556236759; 241.3572026731875 232.26076157621634 424.156405930638; 251.7172796767908 236.241217031755 535.5504130105063; 254.67420019352966 238.16797298974308 541.5103884040872; 258.191247139331 240.28580860163171 548.316096051482; 275.9303925014923 250.1090411990079 579.5571873255813; 277.0137301410879 236.49240657174903 429.2018484794631; 279.1409435572262 238.08462050543042 432.1066064445358; 280.71324348596676 239.24484877398334 434.2029015405541; 282.6103988626051 240.6275780877725 436.53557301227085; 289.41788174283744 245.268237900903 443.40507369696047; 299.022434193066 251.6192335246377 452.0888420459748; 308.9037399467929 258.11581303391836 460.8247632359212; 337.9670488248864 278.22234701007426 490.43578862164713; 264.06095750019944 234.0434030923871 294.07851190801176]

# Local Bubble
R_LB = [212.3, 1.0]
n_LB = [0.494, 0.006]

2-element Vector{Float64}:
 0.494
 0.006

In [54]:
# Create figure
fig = plt.figure(figsize=(8, 8), num=1, clear=true)
ax = gca()

# reduce margins
margins(0,0)

tlim = [2.0, 10.0]
ax.set_xlim(tlim...)
ax.set_ylim(100, 300)

# big tick labels
ax.tick_params(axis="both", labelsize=fontsize)

# add grid
ax.grid(which="major", axis="both", zorder=-1)

# Place Local Bubble in Plot
ax.plot(tlim, fill(R_LB[1], 2), color="purple", linestyle="--", zorder=3, linewidth=2, label="Local Bubble")
ax.fill_between(tlim, fill(R_LB[1] - R_LB[2], 2), fill(R_LB[1] + R_LB[2], 2), color="purple", alpha=0.3, zorder=2, linewidth=0)

# ticks & labels
ax.set_xlabel(L"\mathrm{Time \ [Myr]}", fontsize=fontsize)
ax.set_ylabel(L"\mathrm{Effective \ Radius \ [pc]}", fontsize=fontsize)

# plot simulation data
ax.plot(t_N1, R_N1[:, 1], color=model_colors["N1"], linewidth=2, zorder=2, label="N1")
ax.fill_between(t_N1, R_N1[:, 2], R_N1[:, 3], color=model_colors["N1"], linewidth=0, alpha=0.3, zorder=1)

ax.plot(t_N10, R_N10[:, 1], color=model_colors["N10"], linewidth=2, zorder=2, label="N10")
ax.fill_between(t_N10, R_N10[:, 2], R_N10[:, 3], color=model_colors["N10"], linewidth=0, alpha=0.3, zorder=1)

ax.plot(t_N1x10, R_N1x10[:, 1], color=model_colors["N1x10"], linewidth=2, zorder=2, label="N1x10")
ax.fill_between(t_N1x10, R_N1x10[:, 2], R_N1x10[:, 3], color=model_colors["N1x10"], linewidth=0, alpha=0.3, zorder=1)

# add model lines
p_sf = 2.854e5 # terminal momentum
ax.plot(tlim, (3/π * p_sf * Msol * km_s * tlim * Myr / μ / n_LB[1]).^0.25 / pc, color=model_colors["N1"], linewidth=2, zorder=2, linestyle=":") # N1
ax.plot(tlim, (3/π * p_sf * 10^0.93 * Msol * km_s * tlim * Myr / μ / n_LB[1]).^0.25 / pc, color=model_colors["N10"], linewidth=2, zorder=2, linestyle=":") # N10
ax.plot(tlim, (3/2π * p_sf * Msol * km_s / Myr * (tlim * Myr).^2 / μ / n_LB[1]).^0.25 / pc, color=model_colors["N1x10"], linewidth=2, zorder=2, linestyle=":") # N1x10
ax.plot([NaN], [NaN], color="black", linewidth=2, zorder=2, linestyle=":", label="Model")

# add legend
ax.legend(loc = "upper left", frameon=false, handlelength=1, fontsize=fontsize)

# save figure
savefig(output_directory * "sizes_LB_evolution.pdf", bbox_inches="tight", dpi=300)
fig.clear()
close(fig)

## Figure 2: Solar System Entering Local Bubble

In [56]:
# Create figure
fig = plt.figure(figsize=(8, 8), num=1, clear=true)
ax = gca()

# reduce margins
margins(0,0)

tlim = [0.1, 10.0]
ax.set_xlim(tlim...); ax.set_xscale("log")
ax.set_ylim(30, 420); ax.set_yscale("log")

# big tick labels
ax.tick_params(axis="both", labelsize=fontsize)

# add grid
ax.grid(which="major", axis="both", zorder=-1)

# Solar System
tt = LogRange(log10(tlim[1]), log10(3), 1000)
ax.plot(tt, 20 * km_s * (3 .- tt) * Myr / pc .+ TINY, color="black", linestyle="--", linewidth=2, label=L"\mathrm{t_{age} = 3 \ Myr}")
tt = LogRange(log10(tlim[1]), log10(4), 1000)
ax.plot(tt, 20 * km_s * (4 .- tt) * Myr / pc .+ TINY, color="black", linestyle="--", linewidth=2, label=L"\mathrm{t_{age} = 4 \ Myr}")
tt = LogRange(log10(tlim[1]), log10(5), 1000)
ax.plot(tt, 20 * km_s * (5 .- tt) * Myr / pc .+ TINY, color="black", linestyle="--", linewidth=2, label=L"\mathrm{t_{age} = 5 \ Myr}")
ll.labelLines(ax.get_lines(), align=true, xvals=[1, 1.5, 2], fontsize=0.8fontsize)

# Place Local Bubble in Plot
ax.plot(tlim, fill(R_LB[1], 2), color="purple", linestyle="--", zorder=3, linewidth=2, label="Local Bubble")
ax.fill_between(tlim, fill(R_LB[1] - R_LB[2], 2), fill(R_LB[1] + R_LB[2], 2), color="purple", alpha=0.3, zorder=2, linewidth=0)

# ticks & labels
ax.set_xlabel(L"\mathrm{Time \ [Myr]}", fontsize=fontsize)
ax.set_ylabel(L"\mathrm{Effective \ Radius \ [pc]}", fontsize=fontsize)

# plot simulation data
ax.plot(t_N1, R_N1[:, 1], color=model_colors["N1"], linewidth=2, zorder=2, label="N1")
ax.fill_between(t_N1, R_N1[:, 2], R_N1[:, 3], color=model_colors["N1"], linewidth=0, alpha=0.3, zorder=1)

ax.plot(t_N10, R_N10[:, 1], color=model_colors["N10"], linewidth=2, zorder=2, label="N10")
ax.fill_between(t_N10, R_N10[:, 2], R_N10[:, 3], color=model_colors["N10"], linewidth=0, alpha=0.3, zorder=1)

ax.plot(t_N1x10, R_N1x10[:, 1], color=model_colors["N1x10"], linewidth=2, zorder=2, label="N1x10")
ax.fill_between(t_N1x10, R_N1x10[:, 2], R_N1x10[:, 3], color=model_colors["N1x10"], linewidth=0, alpha=0.3, zorder=1)

# add legend
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[end-3:end], labels[end-3:end], loc = "upper left", frameon=false, handlelength=1, fontsize=fontsize)

# save figure
savefig(output_directory * "solar_system.pdf", bbox_inches="tight", dpi=300)
fig.clear()
close(fig)

## Figure A1: Comparison of Smoothers

In [57]:
# Compute smoothers
W_smooth0       = Gaussian_smoother(r, dV, 9, nbuf=0, mode=:linex, spacing=:log, weighting=:volume);
W_smooth_mirror = Gaussian_smoother(r, dV, 9, nbuf=2, mode=:mirror, spacing=:log, weighting=:volume);
W_smooth_const  = Gaussian_smoother(r, dV, 9, nbuf=2, mode=:const, spacing=:log, weighting=:volume);

In [59]:
# Get radii of buffer region
r_buf = [r[1]^2 / r[i]  for i in 42:-1:2];

In [61]:
# create figure
fig = plt.figure(figsize=(8, 16), num=1, clear=true)

axs = fig.subplots(ncols=1, nrows=2)
subplots_adjust(wspace=0.0, hspace=0.0)

# first axis
ihp = 4489 # direction with peak near boundary

# compute direction in galactic coordinates
θ, ϕ = get_angle(ihp)
l = ϕ * 180 / π
b = 90 - 180 / π * θ

# prepare axes
ax1 = axs[1, 1]
ax1.set_xlim(55, 115)
ax1.set_ylim(3e-2, 1.5); ax1.set_yscale("log")

# plot vertical line, indicating location of boundary
ax1.axvline(r[1], color="darkred", linestyle="--")

# axis labeling
ax1.tick_params(axis="both", labelsize=18)
#ax1.set_xlabel("Distance [pc]", fontsize=18)
ax1.set_ylabel(L"\mathrm{Hydrogen \ Number \ Density \ [cm^{-3}]}", fontsize=18)

# plot profile
ax1.plot(r, profiles_mean[ihp] * Dunit, color="gray", alpha=0.3, linewidth=2)

# plot smoothed profiles
ax1.plot(r, W_smooth0       * profiles_mean[ihp] * Dunit, color="black", linewidth=2, label="No Extrap.") # simply normalize boundaries away
ax1.plot(r, W_smooth_mirror * profiles_mean[ihp] * Dunit, color="red", linewidth=2, label="Refl.") # Reflective Boundaries
ax1.plot(r, W_smooth_const  * profiles_mean[ihp] * Dunit, color="purple", linewidth=2) # Constant Boundaries
ax1.plot(r, W_smooth        * profiles_mean[ihp] * Dunit, color="royalblue", linewidth=2) # Linear Interpolation

# plot buffer region
ρ_buf = profiles_mean[ihp][1] .- [(profiles_mean[ihp][i] - profiles_mean[ihp][1]) * r[1] / r[i] for i in 42:-1:2]
ax1.plot(r_buf, ρ_buf * Dunit, color="royalblue", zorder=3, linewidth=2, alpha=0.3)

ax1.plot(r_buf, fill(profiles_mean[ihp][1], size(r_buf)) * Dunit, color="purple", zorder=3, linewidth=2, alpha=0.3)

ρ_buf = [profiles_mean[ihp][i] for i in 42:-1:2]
ax1.plot(r_buf, ρ_buf * Dunit, color="red", zorder=3, linewidth=2, alpha=0.3)

ax1.legend(loc = "upper right", frameon=false, handlelength=1, fontsize=14, title=L"\ell = %$(round(l, digits=1))^{\circ}, \ b = %$(round(b, digits=1))^{\circ}")

# second axis
ihp = 120551          # Direction with steep gradient

# compute direction in galactic coordinates
θ, ϕ = get_angle(ihp)
l = ϕ * 180 / π
b = 90 - 180 / π * θ

# prepare axes
ax2 = axs[2, 1]
ax2.set_xlim(55, 115)
ax2.set_ylim(1.5e-2, 0.9e-1); ax2.set_yscale("log")

# plot vertical line, indicating location of boundary
ax2.axvline(r[1], color="darkred", linestyle="--")

# axis labeling
ax2.tick_params(axis="both", labelsize=18)
ax2.set_ylabel(L"\mathrm{Hydrogen \ Number \ Density \ [cm^{-3}]}", fontsize=18)
ax2.set_xlabel("Distance [pc]", fontsize=18)

# plot profile
ax2.plot(r, profiles_mean[ihp] * Dunit, color="gray", alpha=0.3, linewidth=2)

# plot smoothed profiles
ax2.plot(r, W_smooth0       * profiles_mean[ihp] * Dunit, color="black", linewidth=2) # simply normalize boundaries away
ax2.plot(r, W_smooth_mirror * profiles_mean[ihp] * Dunit, color="red", linewidth=2) # Reflective Boundaries
ax2.plot(r, W_smooth_const  * profiles_mean[ihp] * Dunit, color="purple", linewidth=2, label="Const.") # Constant Boundaries
ax2.plot(r, W_smooth        * profiles_mean[ihp] * Dunit, color="royalblue", linewidth=2, label="Lin. Extrap.") # Linear Extrapolation

# plot buffer region
ρ_buf = profiles_mean[ihp][1] .- [(profiles_mean[ihp][i] - profiles_mean[ihp][1]) * r[1] / r[i] for i in 42:-1:2]
ax2.plot(r_buf, ρ_buf * Dunit, color="royalblue", zorder=3, linewidth=2, alpha=0.3)
ax2.plot(r_buf, fill(profiles_mean[ihp][1], size(r_buf)) * Dunit, color="purple", zorder=3, linewidth=2, alpha=0.3)
ρ_buf = [profiles_mean[ihp][i] for i in 42:-1:2]
ax2.plot(r_buf, ρ_buf * Dunit, color="red", zorder=3, linewidth=2, alpha=0.3)

ax2.legend(loc = "upper right", frameon=false, handlelength=1, fontsize=14, title=L"\ell = %$(round(l, digits=1))^{\circ}, \ b = %$(round(b, digits=1))^{\circ}")

savefig(output_directory * "smoothing.pdf", bbox_inches="tight", dpi=300)
fig.clear()
close(fig)